### Restart and run all cells

#### csv files: sell-alert

In [1]:
import calendar
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine
from math import ceil, floor, trunc

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
con_lite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 2, 14)

In [2]:
def number_of_spread(x):
    maxp = x[1]
    minp = x[10]
    div = 0.0
    gap = 0.0
    spd = 0
    spd1 = 0
    spd2 = 0
    if (maxp > minp): 
        mult = 1
    else:
    	tmpp = minp
    	minp = maxp
    	maxp = tmpp
    	mult = -1
        
    gap = round(maxp - minp,2)   
    
    if (maxp > 400.00):
        div = 2.0
        boundary = div * 200
        if ((minp < boundary)):
            spd1 = trunc((boundary - minp) / 1.00)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 200.00):
        div = 1.0
        boundary = div * 200
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.50)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 100.00):
        div = 0.50
        boundary = div * 200
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.25)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 25.00):
        div = 0.25
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.10)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div)
    elif (maxp > 10.00):
        div = 0.10
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.05)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            div = 0.10            
            spd = trunc(gap / div)
    elif (maxp > 5.00):
        div = 0.05
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.02)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2           
        else:
            div = 0.05            
            spd = ceil(gap / div)
    elif (maxp > 2.00):
        div = 0.02
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.01)
            spd2 = trunc((maxp - boundary) / div)
            spd = spd1 + spd2
        else:          
            spd = (gap / div)
    else:
        div = 0.01
        boundary = div * 100        
        spd = trunc(gap / div) 
        
    return spd * mult

In [3]:
format_dict = {
               'fm_price':'{:.2f}','to_price':'{:.2f}',    
               'max_price':'{:.2f}','min_price':'{:.2f}',
               'target':'{:.2f}','max':'{:.2f}','max52':'{:.2f}','css_price':'{:.2f}','amount':'{:,.2f}',  
               'unit_cost':'{:.2f}','cost_amt':'{:,.2f}',
               'dividend':'{:.2f}','dividend_amount':'{:,.2f}',    
               'cost_amt':'{:,.2f}','div_amt':'{:,.2f}','cost_pct':'{:.2f}', 
               'buy_target':'{:.2f}','sell_target':'{:.2f}',
               'volume':'{:,.2f}','beta':'{:.2f}','cost':'{:,.2f}',
               'qty':'{:,}','available_qty':'{:,}','shares':'{:,}'}
pd.set_option("display.max_rows", None)

In [4]:
sql = '''
SELECT T.name,sell_target AS target, sh, T.qty,T.available_qty,cost,
A.fm_date, A.to_date, days, fm_price, to_price, ttl_spread AS ttl, 
A.max_price AS max, T.max_price AS max52, consensus_price AS css_price,
sell_target * T.qty AS amount, T.status
FROM stocks T
JOIN sales A
ON T.name = A.name
LEFT JOIN consensus C
ON T.name = C.name
WHERE T.status IN ('I','S')
AND to_date = '%s'
'''
# AND to_price >= fm_price
sql = sql % today
print(sql)

df_stock = pd.read_sql(sql, con_lite)
df_stock.sort_values(['name'],ascending=[True]).style.format(format_dict)


SELECT T.name,sell_target AS target, sh, T.qty,T.available_qty,cost,
A.fm_date, A.to_date, days, fm_price, to_price, ttl_spread AS ttl, 
A.max_price AS max, T.max_price AS max52, consensus_price AS css_price,
sell_target * T.qty AS amount, T.status
FROM stocks T
JOIN sales A
ON T.name = A.name
LEFT JOIN consensus C
ON T.name = C.name
WHERE T.status IN ('I','S')
AND to_date = '2022-02-14'



,name,target,sh,qty,available_qty,cost,fm_date,to_date,days,fm_price,to_price,ttl,max,max52,css_price,amount,status
0,CKP,5.25,0,"10,000","20,000",5.00,2022-02-10,2022-02-14,2,5.20,5.00,-4,5.10,6.30,6.50,"52,500.00",I
1,DIF,14.30,0,"10,000","60,000",14.70,2022-02-08,2022-02-14,4,14.50,14.20,-3,14.30,14.50,15.20,"143,000.00",I
2,GLOBAL,20.40,0,"7,500","15,000",19.40,2022-02-10,2022-02-14,2,20.30,19.80,-5,20.00,24.70,26.45,"153,000.00",I
3,IP,21.20,0,"2,500","7,500",20.20,2022-02-07,2022-02-14,5,22.50,20.10,-24,20.50,24.47,22.20,"53,000.00",I
9,IVL,52.50,12,"1,200","4,800",42.00,2022-02-11,2022-02-14,1,51.50,50.75,-3,51.75,52.00,58.49,"63,000.00",I
10,NER,7.80,0,"9,000","27,000",7.45,2022-02-11,2022-02-14,1,7.20,7.15,-1,7.20,8.65,10.35,"70,200.00",I
8,PTT,41.25,0,"3,000","9,000",38.50,2022-02-11,2022-02-14,1,40.25,40.50,1,41.25,42.50,45.83,"123,750.00",I
4,RCL,44.75,0,"1,500","4,500",44.50,2022-02-10,2022-02-14,2,45.50,41.25,-17,43.50,67.75,41.75,"67,125.00",I
5,TSE,2.62,0,"40,000","40,000",2.60,2022-02-07,2022-02-14,5,2.52,2.48,-2,2.50,3.70,3.43,"104,800.00",I
6,TSTH,1.52,0,"75,000","75,000",1.50,2022-02-10,2022-02-14,2,1.44,1.41,-3,1.43,2.44,1.50,"114,000.00",I


In [5]:
my_array = df_stock.to_numpy()
my_array

array([['CKP', 5.25, 0, 10000, 20000, 5.0, '2022-02-10', '2022-02-14', 2,
        5.2, 5.0, -4, 5.1, 6.3, 6.5, 52500.0, 'I'],
       ['DIF', 14.3, 0, 10000, 60000, 14.7, '2022-02-08', '2022-02-14',
        4, 14.5, 14.2, -3, 14.3, 14.5, 15.2, 143000.0, 'I'],
       ['GLOBAL', 20.4, 0, 7500, 15000, 19.4, '2022-02-10', '2022-02-14',
        2, 20.3, 19.8, -5, 20.0, 24.7, 26.45, 153000.0, 'I'],
       ['IP', 21.2, 0, 2500, 7500, 20.2, '2022-02-07', '2022-02-14', 5,
        22.5, 20.1, -24, 20.5, 24.47, 22.2, 53000.0, 'I'],
       ['RCL', 44.75, 0, 1500, 4500, 44.5, '2022-02-10', '2022-02-14', 2,
        45.5, 41.25, -17, 43.5, 67.75, 41.75, 67125.0, 'I'],
       ['TSE', 2.62, 0, 40000, 40000, 2.6, '2022-02-07', '2022-02-14', 5,
        2.52, 2.48, -2, 2.5, 3.7, 3.43, 104800.0, 'I'],
       ['TSTH', 1.52, 0, 75000, 75000, 1.5, '2022-02-10', '2022-02-14',
        2, 1.44, 1.41, -3, 1.43, 2.44, 1.5, 114000.0, 'I'],
       ['TU', 21.8, 12, 6000, 18000, 20.8, '2022-02-08', '2022-02-14', 4,
   

In [6]:
tmp = (np.apply_along_axis(number_of_spread, axis=1, arr=my_array))
tmp

array([ 5,  1,  5, 11, 14,  7, 11,  8,  3,  7, 13,  9])

In [7]:
df_stock['spd'] = tmp.tolist()
df_stock['spd'] = df_stock['spd'].astype(int)
df_stock.style.format(format_dict)

,name,target,sh,qty,available_qty,cost,fm_date,to_date,days,fm_price,to_price,ttl,max,max52,css_price,amount,status,spd
0,CKP,5.25,0,"10,000","20,000",5.00,2022-02-10,2022-02-14,2,5.20,5.00,-4,5.10,6.30,6.50,"52,500.00",I,5
1,DIF,14.30,0,"10,000","60,000",14.70,2022-02-08,2022-02-14,4,14.50,14.20,-3,14.30,14.50,15.20,"143,000.00",I,1
2,GLOBAL,20.40,0,"7,500","15,000",19.40,2022-02-10,2022-02-14,2,20.30,19.80,-5,20.00,24.70,26.45,"153,000.00",I,5
3,IP,21.20,0,"2,500","7,500",20.20,2022-02-07,2022-02-14,5,22.50,20.10,-24,20.50,24.47,22.20,"53,000.00",I,11
4,RCL,44.75,0,"1,500","4,500",44.50,2022-02-10,2022-02-14,2,45.50,41.25,-17,43.50,67.75,41.75,"67,125.00",I,14
5,TSE,2.62,0,"40,000","40,000",2.60,2022-02-07,2022-02-14,5,2.52,2.48,-2,2.50,3.70,3.43,"104,800.00",I,7
6,TSTH,1.52,0,"75,000","75,000",1.50,2022-02-10,2022-02-14,2,1.44,1.41,-3,1.43,2.44,1.50,"114,000.00",I,11
7,TU,21.80,12,"6,000","18,000",20.80,2022-02-08,2022-02-14,4,21.40,21.00,-4,21.20,23.00,25.75,"130,800.00",I,8
8,PTT,41.25,0,"3,000","9,000",38.50,2022-02-11,2022-02-14,1,40.25,40.50,1,41.25,42.50,45.83,"123,750.00",I,3
9,IVL,52.50,12,"1,200","4,800",42.00,2022-02-11,2022-02-14,1,51.50,50.75,-3,51.75,52.00,58.49,"63,000.00",I,7


In [8]:
sql = '''
SELECT name, volbuy AS shares, price AS unit_cost, volbuy * price AS cost_amt, dividend,
volbuy * dividend AS div_amt, (volbuy * dividend)/(volbuy * price) * 100 AS cost_pct, period
FROM buy
WHERE period IN ("1","2","3","4")
'''
df_buy = pd.read_sql(sql, const)
df_buy['shares'] = df_buy.shares.astype(int)
df_buy.sort_values(['name'],ascending=[True]).style.format(format_dict)

,name,shares,unit_cost,cost_amt,dividend,div_amt,cost_pct,period
9,BCH,"15,000",21.30,"319,500.00",0.33,"4,950.00",1.55,3
27,BGRIM,"9,000",40.00,"360,000.00",0.45,"4,050.00",1.12,4
18,CKP,"20,000",5.00,"100,000.00",0.04,700.00,0.70,3
25,CPNCG,"10,000",12.50,"125,000.00",0.96,"9,587.00",7.67,2
5,DIF,"60,000",14.70,"882,000.00",1.04,"62,640.00",7.10,2
11,DOHOME,"12,000",24.30,"291,600.00",0.10,"1,200.00",0.41,3
7,GLOBAL,"15,000",19.40,"291,000.00",nan,nan,nan,3
16,HREIT,"30,000",8.95,"268,500.00",0.69,"20,796.00",7.75,2
32,IP,"7,500",20.20,"151,500.00",0.18,"1,387.50",0.92,3
2,IVL,"4,800",42.00,"201,600.00",0.85,"4,080.00",2.02,3


In [9]:
df_merge = pd.merge(df_stock, df_buy, on=['name'], how='inner')
cols = 'name spd target sh qty available_qty cost fm_price to_price ttl max max52 css_price status period amount'.split()
df_merge[cols].style.format(format_dict)

,name,spd,target,sh,qty,available_qty,cost,fm_price,to_price,ttl,max,max52,css_price,status,period,amount
0,CKP,5,5.25,0,"10,000","20,000",5.00,5.20,5.00,-4,5.10,6.30,6.50,I,3,"52,500.00"
1,DIF,1,14.30,0,"10,000","60,000",14.70,14.50,14.20,-3,14.30,14.50,15.20,I,2,"143,000.00"
2,GLOBAL,5,20.40,0,"7,500","15,000",19.40,20.30,19.80,-5,20.00,24.70,26.45,I,3,"153,000.00"
3,IP,11,21.20,0,"2,500","7,500",20.20,22.50,20.10,-24,20.50,24.47,22.20,I,3,"53,000.00"
4,RCL,14,44.75,0,"1,500","4,500",44.50,45.50,41.25,-17,43.50,67.75,41.75,I,3,"67,125.00"
5,TSE,7,2.62,0,"40,000","40,000",2.60,2.52,2.48,-2,2.50,3.70,3.43,I,3,"104,800.00"
6,TSTH,11,1.52,0,"75,000","75,000",1.50,1.44,1.41,-3,1.43,2.44,1.50,I,4,"114,000.00"
7,TU,8,21.80,12,"6,000","18,000",20.80,21.40,21.00,-4,21.20,23.00,25.75,I,3,"130,800.00"
8,PTT,3,41.25,0,"3,000","9,000",38.50,40.25,40.50,1,41.25,42.50,45.83,I,3,"123,750.00"
9,IVL,7,52.50,12,"1,200","4,800",42.00,51.50,50.75,-3,51.75,52.00,58.49,I,3,"63,000.00"


In [10]:
sell_alert = df_merge[cols]
sell_alert.set_index(['name'],inplace=True)
sell_alert.sort_values(['spd'],ascending=[True])

,spd,target,sh,qty,available_qty,cost,fm_price,to_price,ttl,max,max52,css_price,status,period,amount
name,,,,,,,,,,,,,,,
DIF,1,14.30,0,10000,60000,14.70,14.50,14.20,-3,14.30,14.50,15.20,I,2,143000.0
PTT,3,41.25,0,3000,9000,38.50,40.25,40.50,1,41.25,42.50,45.83,I,3,123750.0
CKP,5,5.25,0,10000,20000,5.00,5.20,5.00,-4,5.10,6.30,6.50,I,3,52500.0
GLOBAL,5,20.40,0,7500,15000,19.40,20.30,19.80,-5,20.00,24.70,26.45,I,3,153000.0
TSE,7,2.62,0,40000,40000,2.60,2.52,2.48,-2,2.50,3.70,3.43,I,3,104800.0
IVL,7,52.50,12,1200,4800,42.00,51.50,50.75,-3,51.75,52.00,58.49,I,3,63000.0
TU,8,21.80,12,6000,18000,20.80,21.40,21.00,-4,21.20,23.00,25.75,I,3,130800.0
WHART,9,12.80,0,10000,10000,12.50,12.00,11.90,-1,12.00,14.08,14.90,I,4,128000.0
IP,11,21.20,0,2500,7500,20.20,22.50,20.10,-24,20.50,24.47,22.20,I,3,53000.0


In [11]:
file_name = 'sell-alert.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

sell_alert.sort_values(['spd'],ascending=[True]).to_csv(output_file)
sell_alert.sort_values(['spd'],ascending=[True]).to_csv(data_file)
sell_alert.sort_values(['spd'],ascending=[True]).to_csv(box_file)
sell_alert.sort_values(['spd'],ascending=[True]).to_csv(one_file)